# Subset Sum Problem
The [subset sum problem](https://en.wikipedia.org/wiki/Subset_sum_problem) is defined as, given a set of numbers, find a subset which adds up to another number.

## Implementation

For example let us have a set $S$ and a target $T$

$$
T = 7.0
$$

\begin{equation*}
S = 
\begin{bmatrix}
1.0 & 2.0 & 3.0 & 4.0 & 5.0 \\
\end{bmatrix}
\end{equation*}

Our goal is to find a mask $M$, such that, the dot product results in the target. Here is an example of a mask that adds up to our target.

\begin{equation*}
M = 
\begin{bmatrix}
0.0 & 0.0 & 1.0 & 1.0 & 0.0 \\
\end{bmatrix}
\end{equation*}

We can verify that 

$$ T = M \cdot S $$


In [1]:
import tensorflow as tf

In [2]:
@tf.function
def compute_subset_sum(S, M):
    return tf.tensordot(S, M, 1)

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable([0,0,1,1,0],dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    T_ = compute_subset_sum(S, M)
    
print(T_)
print(tape.gradient(T_, S))
print(tape.gradient(T_, M))

tf.Tensor(7.0, shape=(), dtype=float32)
tf.Tensor([0. 0. 1. 1. 0.], shape=(5,), dtype=float32)
tf.Tensor([1. 2. 3. 4. 5.], shape=(5,), dtype=float32)


## Training

However, if we train as is, we find that $M$ is not a mask but it forms a linear combination with its inputs.

In [ ]:
opt = tf.keras.optimizers.Adam(5e-3)

@tf.function
def train_step(S, M, T):
    with tf.GradientTape() as tape:
        T_ = compute_subset_sum(S, M)
        loss = tf.nn.l2_loss(T_ - T)
    
    grads = tape.gradient(loss, M)
    opt.apply_gradients(zip([grads], [M]))
    
    return loss, T_

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable([1,1,1,1,1],dtype=tf.float32)
T = 7

for i in range(1000):
    loss, T_ = train_step(S, M, T)
    if i % 100 == 0:
        actual = compute_subset_sum(S, tf.round(M))
        tf.print(loss, M, T_, actual)

### Bistable loss
To force the values to be close to 0 and 1, we introduce the [Bistable Loss](notebooks/boolean-satisfiability.ipynb)

In [ ]:
from library.loss import bistable_loss

On retraining we find that each element the mask is now closer to 0 or 1

In [ ]:
opt = tf.keras.optimizers.Adam(5e-3)

@tf.function
def train_step(S, M, T):
    with tf.GradientTape() as tape:
        T_ = compute_subset_sum(S, M)
        loss = tf.nn.l2_loss(T_ - T)
        loss += tf.reduce_sum(bistable_loss(M)) * 10
    
    grads = tape.gradient(loss, M)
    opt.apply_gradients(zip([grads], [M]))
    
    return loss, T_

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable([1,1,1,1,1],dtype=tf.float32)
T = 7

for i in range(1000):
    loss, T_ = train_step(S, M, T)
    if i % 100 == 0:
        actual = compute_subset_sum(S, tf.round(M))
        tf.print(loss, M, T_, actual)

## One hot softmax

To further make sure that the mask remains either 0 or 1, we increase the dimentionality of the $M$ and apply softmax along the vertical axis.

\begin{equation*}
M = 
\begin{bmatrix}
0 & 0 & 1 & 1 & 0 \\
\end{bmatrix}
\end{equation*}

becomes

\begin{equation*}
M = 
\begin{bmatrix}
0 & 0 & 1 & 1 & 0 \\
1 & 1 & 0 & 0 & 1 \\
\end{bmatrix}
\end{equation*}

Therefore, $\bar{T}$ becomes

$$ \bar{T} = softmax(M, axis=1) \cdot S $$

In [ ]:
@tf.function
def compute_subset_sum_v2(S, M):
    M = tf.transpose(M)
    return tf.tensordot(S, M[1], 1)

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable(tf.one_hot([0,0,1,1,0], 2),dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    M_s = tf.nn.softmax(M, axis=1)
    T_ = compute_subset_sum_v2(S, M_s)

tf.print(tf.transpose(M))
tf.print(T_)
tf.print(tape.gradient(T_, S))
tf.print(tape.gradient(T_, M))

In [ ]:
opt = tf.keras.optimizers.Adam(5e-3)

@tf.function
def train_step(S, M, T):
    with tf.GradientTape() as tape:
        M_s = tf.nn.softmax(M, axis=1)
        T_ = compute_subset_sum_v2(S, M_s)
        loss = tf.nn.l2_loss(T_ - T)
        loss += tf.reduce_sum(bistable_loss(M_s)) * 10
    
    grads = tape.gradient(loss, M)
    opt.apply_gradients(zip([grads], [M]))
    
    return loss, T_

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable(tf.one_hot([1,1,1,1,1], 2), dtype=tf.float32)
T = 7

for i in range(1000):
    loss, T_ = train_step(S, M, T)
    if i % 100 == 0:
        M_T = tf.transpose(M)
        M_T = tf.nn.softmax(M_T, axis=0)
        actual = compute_subset_sum_v2(S, tf.round(M))
        tf.print(loss, M_T[1], T_, actual)
        
tf.print(M_T)